In [1]:
# Import the necessary libraries
from qiskit import QuantumCircuit, transpile, Aer, execute
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
from qiskit.quantum_info import DensityMatrix
import pickle #For exporting the variables
from tqdm import tqdm

from qiskit import QuantumCircuit
from qiskit.providers.fake_provider import FakePerth
from qiskit import transpile
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.ibmq import IBMQ
from qiskit.providers.aer import AerSimulator

In [4]:
# load ibm-q acc
IBMQ.delete_account()
print(IBMQ.active_account())

IBMQAccountCredentialsNotFound: 'No IBM Quantum Experience credentials found on disk.'

In [6]:
IBMQ.save_account("87702973231459fc9d5e341987cc22103b9dc42f76ef6bce251fc0205e80b2b947da639ca062a8af16b1d2096b4c8ca1250861a96d7d21395fcd67728bf0df89", overwrite=True)

In [7]:
def initialize_all_zeros(nr_qubits):
    # Create a quantum circuit with N qubits
    qc = QuantumCircuit(nr_qubits)
    
    # Initialize all qubits in the |0⟩ state
    for qubit in range(nr_qubits):
        qc.initialize([1, 0], qubit)
    
    return qc

def apply_cnot_chain(qc, control_qubit, nr_qubits):
    # Apply CNOT gates from the control qubit to the list of target qubits
    for target_qubit in range(nr_qubits-1):
        qc.cx(control_qubit, target_qubit+1)

# def random_measurement_basis(nr_qubits): #applies to 1 qubit
#     measurement_basis = [1, 2, 3]
#     return [random.choice(measurement_basis) for _ in range(nr_qubits)]

def measurement_bases_N(nr_qubits):
    # Generate nr_qubits random numbers from the set {x=1, y=2, z=3}
    # random_bases = np.random.choice([1, 2, 3], size=nr_qubits) # X=1 Y=2 Z=3

    random_bases = np.random.choice([1, 2, 3], size=nr_qubits)

    return random_bases



In [9]:
provider = IBMQ.load_account()
print([backend.name() for backend in IBMQ.providers()[0].backends()])

['ibmq_qasm_simulator', 'simulator_statevector', 'simulator_mps', 'simulator_extended_stabilizer', 'simulator_stabilizer', 'ibm_lagos', 'ibm_nairobi', 'ibm_perth', 'ibm_brisbane']


In [10]:
# least busy backend
from qiskit.providers.ibmq import least_busy
backend_free = least_busy(provider.backends(filters= lambda x: not x.configuration().simulator))
print(backend_free)

ibm_nairobi


In [14]:
backend = provider.get_backend('ibm_nairobi')

In [16]:
# backend info for interpretation

config = backend.configuration()
print("This backend is called {0}, and is on version {1}. It has {2} qubit{3}. It "
      "{4} OpenPulse programs. The basis gates supported on this device are {5}."
      "".format(config.backend_name,
                config.backend_version,
                config.n_qubits,
                '' if config.n_qubits == 1 else 's',
                'supports' if config.open_pulse else 'does not support',
                config.basis_gates))

This backend is called ibm_nairobi, and is on version 1.3.3. It has 7 qubits. It does not support OpenPulse programs. The basis gates supported on this device are ['id', 'rz', 'sx', 'x', 'cx', 'reset'].


In [17]:
props = backend.properties()
def describe_qubit(qubit, properties):
    """Print a string describing some of reported properties of the given qubit."""

    # Conversion factors from standard SI units
    us = 1e6
    ns = 1e9
    GHz = 1e-9

    print("Qubit {0} has a \n"
          "  - T1 time of {1} microseconds\n"
          "  - T2 time of {2} microseconds\n"
          "  - U2 gate error of {3}\n"
          "  - U2 gate duration of {4} nanoseconds\n"
          "  - resonant frequency of {5} GHz".format(
              qubit,
              properties.t1(qubit) * us,
              properties.t2(qubit) * us,
              properties.gate_error('sx', qubit),
              properties.gate_length('sx', qubit) * ns,
              properties.frequency(qubit) * GHz))

In [21]:
n_qubits = config.n_qubits
for i in range(n_qubits):
    describe_qubit(i, props)

Qubit 0 has a 
  - T1 time of 62.897228108691934 microseconds
  - T2 time of 29.3308587174565 microseconds
  - U2 gate error of 0.00027313783815884665
  - U2 gate duration of 35.55555555555556 nanoseconds
  - resonant frequency of 5.260485684922551 GHz
Qubit 1 has a 
  - T1 time of 63.781351239657525 microseconds
  - T2 time of 84.38983380886546 microseconds
  - U2 gate error of 0.00026662702376520006
  - U2 gate duration of 35.55555555555556 nanoseconds
  - resonant frequency of 5.17044166224392 GHz
Qubit 2 has a 
  - T1 time of 99.19033222770494 microseconds
  - T2 time of 121.97531392588765 microseconds
  - U2 gate error of 0.00025832009687682246
  - U2 gate duration of 35.55555555555556 nanoseconds
  - resonant frequency of 5.274335093198603 GHz
Qubit 3 has a 
  - T1 time of 109.70778855763636 microseconds
  - T2 time of 56.797368652841996 microseconds
  - U2 gate error of 0.00039261763960023936
  - U2 gate duration of 35.55555555555556 nanoseconds
  - resonant frequency of 5.02659

In [8]:
nr_qubits = 6

n_sample = 20 #amount of times the shadows are generated and measured with 1 shot each

# Initialize a list to store the rows
rows = []

# Initialize an empty list to store the measured result and bases in the correct format for the AI model
measurement_results_in_specific_format = np.zeros((n_sample,nr_qubits), dtype=int)
measurement_bases_in_specific_format = []

for _ in tqdm(range(n_sample)):
    # Create a quantum circuit with N qubits
    qc = initialize_all_zeros(nr_qubits)

    control_qubit = 0 #Usually just the first qubit. Hardcoded because we don't need the control


    # Apply a Hadamard gate to qubit 0
    qc.h(control_qubit)

    apply_cnot_chain(qc, control_qubit, nr_qubits)

    # Choose random measurement basis for each qubit
    measurement_bases = measurement_bases_N(nr_qubits)
    
    # measurement_bases = np.array([1, 1, 1, 3]) #To check a specific case
    # print('measurement_bases', measurement_bases)

    # Store the measurement bases in the list    
    measurement_bases_in_specific_format.append(measurement_bases)
    

    # Apply the measurement bases to the qubits
    for qubit in range(nr_qubits):
        if measurement_bases[qubit] == 1:
            qc.h(qubit)
        elif measurement_bases[qubit] == 2:
            qc.sdg(qubit)
            qc.h(qubit)

    # Add measurements for all qubits in the Z basis
    qc.measure_all()
    


    
    backend_sim = AerSimulator.from_backend(backend)
    
    job = execute(qc, backend_sim, shots=1)
    result = job.result()
    counts = result.get_counts(qc)

    # Iterate through the qubits and add their measurement results to the list
    for qubit in range(nr_qubits):
        basis = measurement_bases[qubit]
        result = int(list(counts.keys())[0][nr_qubits - 1 - qubit])  # Extract the result
        rows.append(pd.DataFrame({"Measurement Basis": [basis], "Measured Result": [result]}))
        
        measurement_results_in_specific_format[_][qubit] = result
    
    # print(measurement_results_in_specific_format[_]) #To compare them to the ones in the paper

    # Concatenate the rows into the DataFrame and reset the index
    df = pd.concat(rows, ignore_index=True)

    

# Display the DataFrame
# print('df',df)

obs_before_tensor = measurement_bases_in_specific_format
# print('obs_before_tensor', obs_before_tensor)

out_before_tensor = [np.array(row) for row in measurement_results_in_specific_format]
# print('out_before_tensor', out_before_tensor)




# Create a dictionary to store your variables
variables_to_export = {
    'obs_before_tensor': obs_before_tensor,
    'out_before_tensor': out_before_tensor
}

# Export the dictionary to a file
with open(f'data/exported_noisy_qubits_{nr_qubits}_samples_{n_sample}.pkl', 'wb') as file:
    pickle.dump(variables_to_export, file)



  0%|                                                    | 0/20 [00:07<?, ?it/s]


QiskitBackendNotFoundError: 'No backend matches the criteria'